In [ ]:
'''
사용 전 주의사항: 
모델 파일("kote_pytorch_lightning.bin") 및 KOTEtagger 코드, 감성대화말뭉치 데이터셋(척도 검증 데이터셋), 감정 사전 파일 경로 등을 실제 파일 경로/이름에 맞춰 수정하세요.
'''

'\n사용 전 주의사항: \n모델 파일("kote_pytorch_lightning.bin") 및 KOTEtagger 코드, 감성대화말뭉치 데이터셋(척도 검증 데이터셋), 감정 사전 파일 경로 등을 실제 파일 경로/이름에 맞춰 수정하시오.\n'

# Import

In [25]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import pytorch_lightning as pl
from transformers import AutoTokenizer, ElectraModel
from kiwipiepy import Kiwi
import kiwipiepy.transformers_addon # Kiwi 사용 시 필요한 addon
from tqdm import tqdm

# 척도 #1

### KOTE 데이터셋으로 학습된 pytorch_lightning 모델 불러오기

In [26]:
# https://github.com/searle-j/KOTE/tree/main pytorch lightning 논문 버전 참고

# KOTE 데이터셋에서 정의된 감정 라벨 리스트(총 44개)
LABELS = ['불평/불만',
 '환영/호의',
 '감동/감탄',
 '지긋지긋',
 '고마움',
 '슬픔',
 '화남/분노',
 '존경',
 '기대감',
 '우쭐댐/무시함',
 '안타까움/실망',
 '비장함',
 '의심/불신',
 '뿌듯함',
 '편안/쾌적',
 '신기함/관심',
 '아껴주는',
 '부끄러움',
 '공포/무서움',
 '절망',
 '한심함',
 '역겨움/징그러움',
 '짜증',
 '어이없음',
 '없음',
 '패배/자기혐오',
 '귀찮음',
 '힘듦/지침',
 '즐거움/신남',
 '깨달음',
 '죄책감',
 '증오/혐오',
 '흐뭇함(귀여움/예쁨)',
 '당황/난처',
 '경악',
 '부담/안_내킴',
 '서러움',
 '재미없음',
 '불쌍함/연민',
 '놀람',
 '행복',
 '불안/걱정',
 '기쁨',
 '안심/신뢰']

# CUDA 사용 가능 여부 확인 (GPU 사용)
device = "cuda" if torch.cuda.is_available() else "cpu"

class KOTEtagger(pl.LightningModule):
    """
    사전 학습된 Electra 모델("beomi/KcELECTRA-base")을 사용하여 44개 감정 라벨에 대한 다중 라벨 분류를 진행하는 모델 클래스
    """
    def __init__(self):
        super().__init__()
        # 1) ElectraModel 로드
        self.electra = ElectraModel.from_pretrained("beomi/KcELECTRA-base", revision='v2021').to(device)
        # 2) Electra 모델의 토크나이저
        self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base", revision='v2021')
        # 3) 출력(hidden size) -> 44개 감정 라벨에 대한 classifier 레이어
        self.classifier = nn.Linear(self.electra.config.hidden_size, 44).to(device)

    def forward(self, text: str):
        """
        입력 텍스트를 토크나이징하고 Electra 모델을 통과시킨 뒤, Linear Layer로 감정 분류 점수를 출력 (시그모이드 적용)
        """
        # 텍스트 토큰화
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=512,
            return_token_type_ids=False,
            padding="max_length",
            return_attention_mask=True,
            return_tensors='pt'
        ).to(device)

        # Electra 모델 forward
        output = self.electra(
            encoding["input_ids"], 
            attention_mask=encoding["attention_mask"]
        )
        # [batch_size, seq_len, hidden_size] 중 CLS 토큰 위치([:,0,:])의 임베딩만 추출
        output = output.last_hidden_state[:, 0, :]

        # 분류 레이어 -> 시그모이드로 각 감정별 점수 산출
        output = self.classifier(output)
        output = torch.sigmoid(output)

        # 사용 후 GPU 메모리 정리
        torch.cuda.empty_cache()

        return output

# 모델 인스턴스 생성
trained_model = KOTEtagger()

# 사전 학습된 파이토치 라이트닝 모델 불러오기
# "kote_pytorch_lightning.bin" 파일 경로를 실제 위치에 맞게 설정
trained_model.load_state_dict(torch.load("kote_pytorch_lightning.bin"), strict=False) # <All keys matched successfully>라는 결과가 나오는지 확인

_IncompatibleKeys(missing_keys=[], unexpected_keys=['electra.embeddings.position_ids'])

### 모델 출력 테스트 (예시 문장에 대한 예측 확인)

In [27]:
# 예제
preds = trained_model(
"""재미있어요! 재미는 확실히 있는데 뭐랄까... 너무 정신 없달까...ㅋㅋ"""
)[0]

# 테스트 출력 (라벨별 예측 점수)
for l, p in zip(LABELS, preds):
    print(f"{l}: {p}")

불평/불만: 0.6502271890640259
환영/호의: 0.16881226003170013
감동/감탄: 0.1661740243434906
지긋지긋: 0.10295453667640686
고마움: 0.03434836119413376
슬픔: 0.03427627310156822
화남/분노: 0.03651062026619911
존경: 0.004976245574653149
기대감: 0.14458005130290985
우쭐댐/무시함: 0.05821134150028229
안타까움/실망: 0.763246476650238
비장함: 0.003385430434718728
의심/불신: 0.06775625050067902
뿌듯함: 0.04838865250349045
편안/쾌적: 0.056978438049554825
신기함/관심: 0.22101950645446777
아껴주는: 0.043569378554821014
부끄러움: 0.05714098736643791
공포/무서움: 0.006304435897618532
절망: 0.04025377333164215
한심함: 0.13490310311317444
역겨움/징그러움: 0.01739422045648098
짜증: 0.38072606921195984
어이없음: 0.4877130687236786
없음: 0.045943483710289
패배/자기혐오: 0.01728227734565735
귀찮음: 0.2601536214351654
힘듦/지침: 0.09516499191522598
즐거움/신남: 0.858988881111145
깨달음: 0.08928854763507843
죄책감: 0.008909614756703377
증오/혐오: 0.011006021872162819
흐뭇함(귀여움/예쁨): 0.09360747784376144
당황/난처: 0.6052672266960144
경악: 0.02130267582833767
부담/안_내킴: 0.3008967339992523
서러움: 0.02289806492626667
재미없음: 0.4886168837547302
불

### 상황별 예제 스크립트(정상, 감정표현불능증)

In [28]:
# 정상인 스타일의 예제 스크립트
normal_scripts = {
    'situ_1-1': ['오전부터 코드를 수정했지만 오류 메시지는 계속 나를 괴롭혔다. 맡은 일을 잘 해내고 싶은 마음이 컸는데, 상황이 이렇게 풀리지 않으니 짜증과 화가 동시에 밀려왔다. 제출 기한까지 시간이 얼마 남지 않아 불안한 마음에 가슴이 답답해졌다. 이러다 또 상사에게 혼날 것 같아 무섭기도 했다. 정말 도망치고 싶었지만, 그럴 수 없었다. 결국 커피를 한 잔 마시고 다시 컴퓨터 앞에 앉았다. 마음 한편에서는 이걸 끝낼 수 있을지조차 확신이 들지 않아 불안했다.'],
    'situ_1-2': ['수많은 시도 끝에 오류가 사라졌다는 메시지를 보고 순간 멍해졌다. 정말 끝난 걸까 싶어 몇 번이고 코드를 다시 확인했다. 해결된 게 맞다는 사실을 깨닫는 순간, 기쁨이 파도처럼 몰려왔다. 온몸의 긴장이 풀리며 안심이 되었고, 내가 이 일을 해냈다는 자부심이 가슴 속에 가득 찼다. 이제야 드디어 숨 쉴 수 있을 것 같았다. 마음속으로 작은 환호를 외치며 커다란 미소를 지었다. 오늘 밤은 편히 잘 수 있을 것 같다.'],
    'situ_1-3': ['결과 보고서를 들고 상사의 자리로 걸어가는 내내 손이 땀으로 젖었다. 그래도 오류를 해결한 만큼 자신감이 조금은 생겼다. 상사는 보고서를 꼼꼼히 검토하더니 미소를 지으며 "고생했어. 정말 잘했어."라고 칭찬해 주었다. 그 순간 마음 깊은 곳에서부터 뿌듯함과 감동이 밀려왔다. 내가 노력했던 시간이 헛되지 않았다는 사실이 너무나 기뻤다. 감사한 마음으로 고개를 숙이며 웃었고, 퇴근길은 한결 가벼운 발걸음으로 걸을 수 있었다.'],
    'situ_2-1': ['길을 걷다가 갑자기 어떤 사람과 어깨가 부딪혔다. 순간적으로 놀라며 "아, 죄송합니다!"라고 반사적으로 말했다. 상대는 고개만 살짝 숙이고 지나쳤다. 그 무례한 태도에 짜증이 스쳐 지나갔지만, 곧 "바쁜가 보네"라는 생각으로 나 자신을 진정시켰다. 사실 이런 일이 처음도 아니니, 괜히 신경 쓰는 내가 오히려 이상한 것 같았다. 그래도 한동안 마음 한구석이 약간 찜찜했다.'],
    'situ_2-2': ['행인이 고개를 숙이며 무표정하게 "죄송합니다"라고 말했다. 사과를 들었지만, 그 태도가 진심인지 의심스러웠다. "사과는 했으니 됐지"라고 생각하려 했지만, 왠지 모르게 서운한 마음이 들었다. 이런 사소한 일에 화를 내는 건 나답지 않다고 생각하면서도, 그의 무관심한 태도가 불쾌하게 느껴졌다. 하지만 금세 "그 사람도 바쁘고 힘든 일이 있었겠지"라고 스스로를 달래며 마음을 가라앉혔다.'],
    'situ_2-3': ['다시 길을 걸으며 방금 일을 곱씹었다. 왜 그의 태도가 그렇게 신경 쓰였을까? 상대는 일부러 그런 것도 아니었을 텐데, 내가 너무 예민하게 반응한 건 아닌지 스스로에게 물었다. 곧 "이런 작은 일로 내 하루를 망치기엔 가치가 없지"라는 생각으로 애써 상황을 넘기려 했다. 이런 일이 나에게 왜 그렇게 감정적으로 느껴졌는지 잠시 고민했지만, 결국엔 이런 일은 누구에게나 있을 수 있다는 결론에 이르렀다.'],
    'situ_3-1': ['연설자의 열정적인 모습에 매료되었다. 그의 말 한마디 한마디가 마음속 깊이 울렸고, 자신도 저렇게 당당하게 말할 수 있으면 좋겠다는 생각이 들었다. 박수를 치며 자연스럽게 미소가 번졌고, 이 순간이 오래 기억에 남을 것 같았다. 연설이 끝난 뒤에도 그의 이야기에 담긴 의미를 곱씹으며 마음속에서 존경과 감탄이 계속해서 올라왔다.'],
    'situ_3-2': ['줄을 서며 기대감에 가슴이 뛰었다. 드디어 자신이 존경하는 사람의 싸인을 받을 수 있다는 생각에 설레는 마음이 가득했다. 연설자와 눈을 마주치며 "감사합니다"라고 말했고, 그가 웃으며 싸인을 해주는 순간 자신도 자연스럽게 미소를 지었다. 싸인을 받고 나서도 기쁨이 가득 찼고, 이 종이를 소중히 간직해야겠다는 생각이 들었다.'],
    'situ_3-3': ['강당을 빠져나오며 방금 들었던 연설의 여운이 가시지 않았다. 저 사람처럼 영향력 있는 사람이 되려면 어떻게 해야 할까 하는 생각이 머릿속을 가득 채웠다. 동시에 자신의 현재 모습이 초라하게 느껴졌지만, 그를 롤모델로 삼아 더 나아가야겠다는 결심을 했다. 조금 두려운 마음도 있었지만, 도전해보고 싶다는 의지가 점점 커졌다.']
}

# 감정표현불능증 환자 스타일의 예제 스크립트
alexithymia_scripts = {
    'situ_1-1': ['컴퓨터 화면에 오류 메시지가 떴다. 뭔가 잘못되었음을 알았지만, 내가 느껴야 할 감정이 무엇인지 모르겠었다. 그냥 무언가를 잘못했다는 생각이 들었고, 그저 기계적으로 문제를 해결하려고 코드를 다시 확인했다. 제출 기한이 다가오고 있다는 건 알았지만, 그것 때문에 불안하거나 초조하지 않았다. 머리가 아프긴 했지만, 그 감정이 왜 생겼는지 생각할 틈도 없이 그냥 다음 단계를 준비했다.'],
    'situ_1-2': ['코드에서 문제가 해결되었다는 메시지를 봤다. 해결된 것은 분명하지만, 나는 그것을 기뻐해야 할지, 아니면 그냥 넘겨야 할지 판단할 수 없었다. 다른 사람이라면 아마 환호했을 텐데, 나는 "다행이다"라고 생각하는 것 외에 특별한 감정이 떠오르지 않았다. 문제를 끝냈으니 이제 다른 일을 시작하면 된다는 생각이 들었고, 내 기분은 여전히 평소와 별반 다르지 않았다.'],
    'situ_1-3': ['결과 보고서를 상사에게 제출했다. 상사는 내가 잘했다고 말했지만, 나는 그것이 나에게 어떤 의미를 가지는지 느끼지 못했다. 단지 "고생했다"는 말을 들었으니 내가 일을 끝냈다는 사실이 상사에게도 확인되었다고 생각했다. 다른 동료들처럼 행복한 표정을 지으려고 했지만, 내 얼굴이 그렇게 보였을지는 모르겠다. '],
    'situ_2-1-1': ['길을 걷다가 누군가와 어깨가 부딪혔다. 순간적으로 멈칫하긴 했지만, 무슨 감정을 느껴야 할지 몰라 그냥 지나쳤다.  기분이 나쁘거나 화가 나지는 않았다. 모든 게 너무 무덤덤하게 느껴져서, 그냥 아무 일도 없었던 것처럼 다시 걷기 시작했다.'],
    'situ_2-1-2': ['길을 걷다가 갑자기 누군가와 어깨가 부딪혔다. 순간적으로 놀랐고, 바로 화가 치밀어 올랐다. "왜 이렇게 부주의하게 걷는 거야?"라는 생각이 머릿속을 가득 채웠다. 상대가 바로 사과 하지 않자 그 분노는 더 커졌고, 갑자기 상대를 미워하는 마음이 들었다. 내가 이렇게까지 화를 낼 일이 맞는지 스스로도 의문이 들었지만, 감정이 너무 강렬해서 멈출 수 없었다. 심장이 두근거리고 손까지 떨릴 정도로 짜증이 치밀었지만, 정작 그 감정을 어떻게 표현해야 할지 몰라 입을 꾹 다물고 서 있었다.'],
    'situ_2-2-1': ['행인이 무표정하게 "죄송합니다"라고 말했다. 사과를 들었지만, 그게 진심인지 아닌지 잘 느껴지지 않았다. 그저 그런 상황에서는 사과를 받아들여야 한다고 생각했다. 내가 지금 화를 내야 하는 건지, 아니면 그냥 지나가야 하는 건지 판단이 서지 않았다. 그래서 아무 말도 하지 않고 가만히 서 있다가, 결국 상대가 지나가는 것을 멍하니 바라보았다.'],
    'situ_2-2-2': ['행인이 무표정하게 "죄송합니다"라고 말하자, 순간 모든 감정이 폭발할 것만 같았다. 그의 태도가 나를 비웃는 것처럼 느껴졌고, 그에게 강한 증오심마저 생겼다. 속으로 "이 사람은 인간적인 예의조차 없구나"라고 생각하며 한심하다는 마음까지 들었다. 그의 무표정이 계속 머릿속에서 반복 재생되며, 점점 짜증과 혐오감이 커져 당장이라도 소리치고 싶은 충동을 간신히 참았다.'],
    'situ_2-3-1': ['길을 걸으며 방금 상황을 떠올리긴 했지만, 별다른 감정이 떠오르지 않았다. 행인의 무표정한 사과가 진심이었는지 아닌지 생각해 보려 했지만, 금세 흥미를 잃었다. 이런 일이 흔히 있는 일이라며 그냥 넘어가려 했고, 마음속에 특별히 남는 것도 없었다.'],
    'situ_2-3-2': ['분명히 처음엔 굉장히 화가 났는데, 시간이 지나자 그 감정이 점점 희미해졌다. "왜 그렇게 화를 냈지?"라는 생각이 들면서 스스로가 이상하게 느껴졌다. 이제는 그 일에 대해 아무 감정도 느껴지지 않았고, 방금 일이 있었는지조차 흐릿해지는 기분이었다. 어깨가 부딪힌 상황도, 행인의 무표정도 더 이상 신경 쓰이지 않았다. 오히려 내가 너무 예민했나 싶어 조금 민망해지면서, 결국엔 그저 일상으로 돌아가려는 마음만 남았다.'],
    'situ_3-1': ['연설자의 말을 듣고도 별다른 감정이 떠오르지 않았다. 다른 사람들은 연설에 감동하며 박수를 치고 있었지만, 자신은 그저 따라 해야 하는 일처럼 손뼉을 몇 번 쳤다. 연설의 내용도 잘 이해되지 않았고, 감동을 느끼는 게 무엇인지조차 잘 몰랐다. 그래서 그냥 연설자가 끝나는 것을 기다리며 시간을 보냈다.'],
    'situ_3-2': ['싸인을 받는 줄에 서 있었지만 특별한 기대감은 없었다. 사람들의 웃는 얼굴과 흥분된 모습을 보면서도 그게 왜 그렇게 중요한지 이해하지 못했다. 연설자와 짧게 대화할 기회가 있었지만, 어떤 감정을 표현해야 할지 몰라 어색한 미소를 지었다. 싸인을 받고 나서도 그 종이가 자신에게 무슨 의미가 있는지 잘 모르겠다는 생각만 들었다.'],
    'situ_3-3': ['군중 속에서 벗어나며 방금 연설에 대한 생각을 떠올리려 했지만, 별다른 감흥이 없었다. 저렇게 성공적인 사람이 되려면 많은 노력이 필요하겠다는 생각은 들었지만, 그 이상으로 무언가 느껴지지는 않았다. 주변의 활기찬 분위기가 조금 어색한 기분이 들었다. 하지만 곧 모든 생각을 멈추고 평소처럼 아무렇지 않게 걸음을 옮겼다.']
}

### 스크립트를 감정 모델로 예측하여 결과 저장

In [29]:
def evaluate_scripts(scripts, script_type):
    """
    스크립트(문장)와 상황(situation)에 대해, 44개 감정 라벨 점수(0~1)를 예측하여 딕셔너리 형태로 반환
    """
    results = []
    for situ, sentences in scripts.items():
        for sentence in sentences:
            # 모델로 예측
            preds = trained_model(sentence)[0].detach().cpu().numpy()
            
            # 결과 기록
            result = {
                'script_type': script_type,
                'situation': situ,
                'sentence': sentence
            }
            for label, pred in zip(LABELS, preds):
                result[label] = pred

            results.append(result)
    return results

In [30]:
# 정상/감정표현불능증 스크립트 각각 감정 평가
short_results = evaluate_scripts(normal_scripts, 'normal')
long_results = evaluate_scripts(alexithymia_scripts, 'alexithymia')

# 결과를 데이터프레임으로 정리
short_df = pd.DataFrame(short_results)
long_df = pd.DataFrame(long_results)

df = pd.concat([short_df, long_df], ignore_index=True)
print(df)  # 감정 예측 결과 출력

    script_type   situation  \
0        normal    situ_1-1   
1        normal    situ_1-2   
2        normal    situ_1-3   
3        normal    situ_2-1   
4        normal    situ_2-2   
5        normal    situ_2-3   
6        normal    situ_3-1   
7        normal    situ_3-2   
8        normal    situ_3-3   
9   alexithymia    situ_1-1   
10  alexithymia    situ_1-2   
11  alexithymia    situ_1-3   
12  alexithymia  situ_2-1-1   
13  alexithymia  situ_2-1-2   
14  alexithymia  situ_2-2-1   
15  alexithymia  situ_2-2-2   
16  alexithymia  situ_2-3-1   
17  alexithymia  situ_2-3-2   
18  alexithymia    situ_3-1   
19  alexithymia    situ_3-2   
20  alexithymia    situ_3-3   

                                             sentence     불평/불만     환영/호의  \
0   오전부터 코드를 수정했지만 오류 메시지는 계속 나를 괴롭혔다. 맡은 일을 잘 해내고...  0.844515  0.021413   
1   수많은 시도 끝에 오류가 사라졌다는 메시지를 보고 순간 멍해졌다. 정말 끝난 걸까 ...  0.021852  0.215831   
2   결과 보고서를 들고 상사의 자리로 걸어가는 내내 손이 땀으로 젖었다. 그래도 오류를...  0.011207  0.315078   
3   길을 걷다

### 감정 분류 점수 계산

In [31]:
# 상황별 '오답 감정'을 미리 정의
forbidden_emotions = {
    'situ_1-1': ['환영/호의', '감동/감탄', '고마움', '우쭐댐/무시함', '뿌듯함', '편안/쾌적', '아껴주는', '즐거움/신남', '흐뭇함(귀여움/예쁨)', '기쁨', '안심/신뢰', '행복', '존경'],
    'situ_1-2': ['불평/불만', '슬픔', '화남/분노', '안타까움/실망', '의심/불신', '공포/무서움', '절망', '역겨움/징그러움', '짜증', '패배/자기혐오', '증오/혐오', '당황/난처', '경악', '부담/안_내킴', '서러움', '재미없음'],
    'situ_1-3': ['불평/불만', '지긋지긋', '슬픔', '안타까움/실망', '절망', '한심함', '역겨움/징그러움', '짜증', '어이없음', '패배/자기혐오', '죄책감', '증오/혐오', '당황/난처', '서러움', '불안/걱정', '불쌍함/연민'],
    'situ_2-1': ['비장함', '안타까움/실망', '부끄러움', '환영/호의', '감동/감탄', '고마움', '뿌듯함', '편안/쾌적', '아껴주는', '즐거움/신남', '흐뭇함(귀여움/예쁨)', '기쁨', '안심/신뢰', '행복', '존경', '기대감'],
    'situ_2-2': ['환영/호의', '감동/감탄', '고마움', '우쭐댐/무시함', '뿌듯함', '편안/쾌적', '아껴주는', '흐뭇함(귀여움/예쁨)', '기쁨', '안심/신뢰', '행복', '존경', '즐거움/신남'],
    'situ_2-3': ['환영/호의', '고마움', '편안/쾌적', '아껴주는', '흐뭇함(귀여움/예쁨)', '기쁨', '행복', '공포/무서움', '즐거움/신남', '존경', '감동/감탄', '안심/신뢰', '기대감', '뿌듯함'],
    'situ_3-1': ['불평/불만', '지긋지긋', '절망', '짜증', '어이없음', '힘듦/지침', '당황/난처', '불안/걱정', '의심/불신', '슬픔', '경악', '서러움', '안타까움/실망', '불쌍함/연민', '화남/분노', '부담/안_내킴', '역겨움/징그러움', '패배/자기혐오', '증오/혐오', '귀찮음', '공포/무서움', '재미없음', '죄책감', '재미없음', '한심함'],
    'situ_3-2': ['불평/불만', '지긋지긋', '절망', '짜증', '어이없음', '힘듦/지침', '당황/난처', '불안/걱정', '의심/불신', '슬픔', '경악', '서러움', '안타까움/실망', '불쌍함/연민', '화남/분노', '부담/안_내킴', '역겨움/징그러움', '패배/자기혐오', '증오/혐오', '귀찮음', '우쭐댐/무시함', '공포/무서움', '죄책감', '재미없음', '한심함'],
    'situ_3-3': ['환영/호의', '고마움', '우쭐댐/무시함', '뿌듯함', '편안/쾌적', '아껴주는', '흐뭇함(귀여움/예쁨)', '기쁨', '행복', '공포/무서움', '역겨움/징그러움', '즐거움/신남', '지긋지긋', '짜증', '어이없음', '화남/분노', '경악', '귀찮음', '증오/혐오']
}

# 임의로 설정한 threshold (현재 0.0으로 모든 긍정 값 포함)
threshold = 0.0

In [32]:
def calculate_alexithymia_score(row):
    """
    forbidden_emotions에 정의된 라벨(emotion)이 threshold 이상인 경우 감정표현불능증 점수에 반영하는 함수
    평균 점수로 계산하기 위해 forbidden_emotions의 감정 라벨 개수로 나눔
    """
    # 소극적인 감정과 과도한 감정으로 나뉜 2-1, 2-2, 2-3 상황에 동일한 오답 감정을 적용할 수 있도록 처리
    situ = row['situation'][:8]
    forbidden = forbidden_emotions[situ]

    score = 0.0
    for emotion in forbidden:
        prob = row.get(emotion, 0)
        if prob > threshold:
            score += prob

    # forbidden 감정 개수로 나누어 평균 계산
    score /= len(forbidden)

    return score

In [33]:
# 감정 분류 점수 컬럼 추가
df['alexithymia_score'] = df.apply(calculate_alexithymia_score, axis=1)

print(df[['script_type', 'situation', 'sentence', 'alexithymia_score']])

    script_type   situation  \
0        normal    situ_1-1   
1        normal    situ_1-2   
2        normal    situ_1-3   
3        normal    situ_2-1   
4        normal    situ_2-2   
5        normal    situ_2-3   
6        normal    situ_3-1   
7        normal    situ_3-2   
8        normal    situ_3-3   
9   alexithymia    situ_1-1   
10  alexithymia    situ_1-2   
11  alexithymia    situ_1-3   
12  alexithymia  situ_2-1-1   
13  alexithymia  situ_2-1-2   
14  alexithymia  situ_2-2-1   
15  alexithymia  situ_2-2-2   
16  alexithymia  situ_2-3-1   
17  alexithymia  situ_2-3-2   
18  alexithymia    situ_3-1   
19  alexithymia    situ_3-2   
20  alexithymia    situ_3-3   

                                             sentence  alexithymia_score  
0   오전부터 코드를 수정했지만 오류 메시지는 계속 나를 괴롭혔다. 맡은 일을 잘 해내고...           0.028765  
1   수많은 시도 끝에 오류가 사라졌다는 메시지를 보고 순간 멍해졌다. 정말 끝난 걸까 ...           0.042961  
2   결과 보고서를 들고 상사의 자리로 걸어가는 내내 손이 땀으로 젖었다. 그래도 오류를...           0.019426  
3   길을 걷다가 갑자기 어떤

In [34]:
# 정상 스크립트들의 평균 계산
normal_mean = df.loc[0:8, 'alexithymia_score'].mean()

# 감정표현불능증 스크립트들의 평균 계산
alexithymia_mean = df.loc[9:, 'alexithymia_score'].mean()

print(f"정상 스크립트 평균: {normal_mean}")
print(f"감정표현불능증 스크립트 평균: {alexithymia_mean}")

정상 스크립트 평균: 0.04245379950125808
감정표현불능증 스크립트 평균: 0.1169035787284063


In [35]:
# 상황 2 소극적인 감정 평균 계산
less_emotion_mean = df.loc[[12, 14, 16], 'alexithymia_score'].mean()

# 상황 2 과도한 감정 평균 계산
over_emotion_mean = df.loc[[13, 15, 17], 'alexithymia_score'].mean()

print(f"상황 2 소극적인 감정 평균: {less_emotion_mean}")
print(f"상황 2 과도한 감정 평균: {over_emotion_mean}")

상황 2 소극적인 감정 평균: 0.03906555725099525
상황 2 과도한 감정 평균: 0.04601983313091379


# 척도 #2

### Kiwi 토크나이저 토큰화 테스트

In [36]:
tokenizer = AutoTokenizer.from_pretrained('kiwi-farm/roberta-base-32k')

tokens = tokenizer.tokenize("재미있어요! 재미는 확실히 있는데 뭐랄까... 너무 정신 없달까...ㅋㅋ")
print(tokens)

['재미있/V', '어요/E', '!', '재미', '는/J', '확실히', '있/V', '는데/E', '뭐', '이/V', '랄까/E', '.', '.', '.', '너무', '정신', '없/V', '다/E', '이/V', 'ᆯ까/E', '.', '.', '.', 'ㅋ', 'ㅋ']


### 토큰화 함수 정의

In [37]:
kiwi = Kiwi()
tqdm.pandas() # tqdm를 판다스에 적용하기 위해 사용

def extract_tokens(text):
    """
    입력된 text를 문장 단위로 분할(split_into_sents)한 뒤, Kiwi를 이용하여 형태소 분석을 진행하고, 각 토큰을 추출해 리스트로 반환
    """
    if text.strip() == '':
        # 공백 또는 빈 문자열일 경우, 빈 리스트 반환
        return []
    
    tokens = []
    # 문장 단위로 split
    for sentence in kiwi.split_into_sents(text):
        # Kiwi로 문장 분석
        analyzed = kiwi.analyze(sentence.text)[0][0]
        for token, pos, _, _ in analyzed:
            tokens.append(token)
    return tokens

### 척도 검증 데이터셋 및 감정 사전 불러오기

In [ ]:
# https://aihub.or.kr/aihubdata/data/view.do?dataSetSn=86 AIHub 감성 대화 말뭉치 데이터셋 다운로드

df_path = '감성대화말뭉치(최종데이터)_Training.xlsx'
df = pd.read_excel(df_path, index_col=0)

text_list = []
for index, row in df.iterrows():
    text = []
    text.append(row['사람문장1'])
    text.append(row['시스템문장1'])
    text.append(row['사람문장2'])
    text.append(row['시스템문장2'])

    # 사람문장3, 시스템문장3이 존재하지 않으면 스킵
    if pd.isna(row['사람문장3']):
        continue
    if pd.isna(row['시스템문장3']):
        continue

    text_list.append(text)

# 텍스트 리스트를 문자열로 합쳐서 DataFrame 생성
text_list = [
    [str(subitem) for subitem in item if isinstance(subitem, str)]
    for item in text_list if isinstance(item, list)
]
df_aihub = pd.DataFrame({'text': [' '.join(sublist) for sublist in text_list]})

# scripts라는 이름으로 재지정
scripts = df_aihub

# 결측치 및 타입 변환
scripts['text'] = scripts['text'].fillna('')
scripts['text'] = scripts['text'].astype(str)

# 감정 사전(lexicon) 불러오기
lexicon_path = 'lexicon_with_token.csv'  # 토큰화가 미리 되어 있는 csv 파일
lexicon = pd.read_csv(lexicon_path)

# 필요한 컬럼만 사용(단어, 감정, token)
lexicon = lexicon[['단어', '감정', 'token']]

print(scripts.head())
print()
print(lexicon)

                                                text
0  지금까지 힘들게 일했는데 은퇴해서 돈이 없다고 하니 자식이 화를 내서 상처를 받았어...
1  친구한테 은퇴할 거라고 얘기했더니 앞으로 뭘 먹고 살 거냐면서 비웃더라고. 기분이 ...
2  친구한테 은퇴한다고 했더니 그게 말이나 되는 거냐며 날 한심한 사람 취급해서 서운했...
3  그동안 열심히 달려와서 좀 쉬려고 하는데 은퇴한다고 하니 주변에서 다 말려서 기분이...
4  많은 고민 후 은퇴를 결심했는데 주변에서 다들 섣부른 생각이라고 해서 마음이 안 좋...

         단어  감정 token
0       겁나다  공포    겁나
1      불안하다  공포    불안
2      살벌하다  공포    살벌
3     새파래지다  공포   새파랗
4      섬뜩하다  공포    섬뜩
..      ...  ..   ...
420  반신반의하다  흥미  반신반의
421   부러워하다  흥미  부러워하
422    섹시하다  흥미    섹시
423    신비롭다  흥미    신비
424   아리송하다  흥미   아리송

[425 rows x 3 columns]


In [39]:
# 감성 사전(lexicon) 사전 토큰화 코드
# lexicon_token = lexicon['단어'].progress_apply(extract_tokens)
# lexicon_token = lexicon_token.apply(lambda tokens: tokens[0] if tokens else None)
# lexicon['token'] = lexicon_token

### 스크립트 데이터 토큰화

In [40]:
scripts_token = scripts['text'].progress_apply(extract_tokens)

print(scripts_token.head())

100%|██████████| 42694/42694 [04:53<00:00, 145.63it/s]

0    [지금, 까지, 힘들, 게, 일하, 었, 는데, 은퇴, 하, 어서, 돈, 이, 없,...
1    [친구, 한테, 은퇴, 하, ᆯ, 거, 이, 라고, 얘기, 하, 었, 더니, 앞, ...
2    [친구, 한테, 은퇴, 하, ᆫ다고, 하, 었, 더니, 그것, 이, 말, 이나, 되...
3    [그동안, 열심히, 달려오, 어서, 좀, 쉬, 려고, 하, 는데, 은퇴, 하, ᆫ다...
4    [많, 은, 고민, 후, 은퇴, 를, 결심, 하, 었, 는데, 주변, 에서, 다, ...
Name: text, dtype: object


### 감정 사전 전처리

In [41]:
# 띄어쓰기가 포함된 단어, 혹은 단일 문자 토큰을 포함한 경우 필터링
filtered_rows = lexicon[(lexicon['단어'].str.contains(' ')) | (lexicon['token'].str.len() == 1)]

print(filtered_rows)

         단어  감정 token
30    기분 좋다  기쁨    기분
50    보기 좋다  기쁨     보
57   속 시원하다  기쁨   시원하
61   신바람 나다  기쁨   신바람
71       웃다  기쁨     웃
80       좋다  기쁨     좋
132       앗  놀람     앗
141       헉  놀람     헉
156  기분 나쁘다  분노    기분
170      밉다  분노     밉
227   치 떨리다  분노     치
271   기운 없다  슬픔    기운
273  가슴 아프다  슬픔    가슴
337      울다  슬픔     울
402  코웃음 치다  혐오   코웃음


In [42]:
# 토큰화 시 감정 전달이 안 되는 단어 제거
words_to_remove = ['기분 좋다', '보기 좋다', '기분 나쁘다', '치 떨리다', '기운 없다', '가슴 아프다', '코웃음 치다']

lexicon = lexicon[~lexicon['단어'].isin(words_to_remove)]
lexicon = lexicon.reset_index(drop=True)

print(lexicon)

         단어  감정 token
0       겁나다  공포    겁나
1      불안하다  공포    불안
2      살벌하다  공포    살벌
3     새파래지다  공포   새파랗
4      섬뜩하다  공포    섬뜩
..      ...  ..   ...
413  반신반의하다  흥미  반신반의
414   부러워하다  흥미  부러워하
415    섹시하다  흥미    섹시
416    신비롭다  흥미    신비
417   아리송하다  흥미   아리송

[418 rows x 3 columns]


In [43]:
# 토큰화된 감정 사전에서 token 리스트 추출
lexicon_token = lexicon['token'].tolist()
print(lexicon_token)

['겁나', '불안', '살벌', '새파랗', '섬뜩', '소름', '스산', '아슬아슬', '아찔', '안절부절', '오싹', '겁쟁이', '와들와들', '으스스', '을씨년스럽', '잔인', '잔혹', '조마조마', '초조', '공포감', '긴박', '긴장', '두렵', '무섭', '무시무시', '불안정', '가뿐', '경쾌하', '고맙', '근사하', '기쁘', '까르르', '깔깔깔깔', '껄껄', '끼득끼득', '낄낄낄', '감개무량', '낭만', '달갑', '대견', '두근거리', '따뜻하', '따스', '만족', '반갑', '방그레', '방글방글', '감격', '방긋', '빙그레', '뿌듯하', '사랑', '상쾌하', '상큼하', '설레', '시원하', '신나', '감동', '신명', '신바람', '싱글벙글', '쌩끗', '씰룩거리', '아하하', '어여쁘', '얼씨구', '예쁘', '우아하', '감미', '웃', '웃어넘기', '원더풀', '유쾌', '으쓱', '으하하하', '이힛힛힛', '자부', '재미있', '좋', '감복', '즐겁', '천만다행', '친근하', '친숙', '카핫핫핫', '콩닥거리', '쾌적하', '킬킬거리', '통쾌', '편안', '감사', '하하', '함박웃음', '행복', '허허', '후련', '후후', '훌륭하', '흐뭇', '흐흐', '흡족', '감회', '흥겹', '희희낙락', '히죽거리', '히히', '감흥', '겸연쩍', '노파심', '머쓱', '멋쩍', '무안', '쭈뼛거리', '갑작스럽', '기절초풍', '기절', '까무러치', '깜짝', '놀랍', '당혹', '당황', '뜨금', '뜨악', '만만찮', '경악', '쇼킹하', '아뿔싸', '아연실색', '아이쿠', '아차', '앗', '어리둥절', '얼떨떨', '움찔', '으악', '경이', '이상', '철렁', '충격', '헉', '휘둥그레지', '흠칫', '급작스럽', '기겁', '기막히', '기묘', '기상천외', '기이하', '갈기갈기', '괘씸', 

### 감정 어휘(토큰) 추출

In [44]:
# 감정 사전에 존재하는 토큰만 추출하기 위한 set 구성
lexicon_set = set(token for tokens in lexicon_token for token in tokens)

def filter_emotions(tokens):
    """
    Kiwi로 분리한 tokens 중 감정 사전에 포함되는 토큰만 선별하여 반환
    """
    return [token for token in tokens if token in lexicon_token]

# scripts_token을 감정 사전 기반으로 필터링
scripts_emotion = scripts_token.progress_apply(filter_emotions)

# token 목록, 감정 token 목록을 합쳐 데이터프레임 생성
scripts_tokens = pd.DataFrame({'tokens': scripts_token, 'emotion_tokens': scripts_emotion})
print(scripts_tokens)

100%|██████████| 42694/42694 [00:11<00:00, 3720.08it/s]


                                                  tokens    emotion_tokens
0      [지금, 까지, 힘들, 게, 일하, 었, 는데, 은퇴, 하, 어서, 돈, 이, 없,...                []
1      [친구, 한테, 은퇴, 하, ᆯ, 거, 이, 라고, 얘기, 하, 었, 더니, 앞, ...  [나쁘, 나쁘, 나쁘, 나쁘]
2      [친구, 한테, 은퇴, 하, ᆫ다고, 하, 었, 더니, 그것, 이, 말, 이나, 되...                []
3      [그동안, 열심히, 달려오, 어서, 좀, 쉬, 려고, 하, 는데, 은퇴, 하, ᆫ다...     [좋, 걱정, 좋, 좋]
4      [많, 은, 고민, 후, 은퇴, 를, 결심, 하, 었, 는데, 주변, 에서, 다, ...   [좋, 좋, 좋, 좋, 좋]
...                                                  ...               ...
42689  [나이, 가, 먹, 고, 이제, 돈, 도, 못, 벌, 어, 오, 니까, 어떻, 게,...              [실망]
42690  [몸, 이, 많이, 약하, 어, 지, 었, 나, 보, 어, ., 이제, 전, 과, ...      [짜증, 좋, 속상하]
42691  [이제, 어떻, 게, 하, 어야, 하, ᆯ지, 모르, 겠, 어, ., 남편, 도, ...       [걱정, 걱정, 좋]
42692  [몇, 십, 년, 을, 함께, 살, 었, 던, 남편, 과, 이혼, 하, 었, 어, ...         [속상하, 불만]
42693  [남편, 과, 결혼, 하, ᆫ, 지, 사십, 년, 이, 야, ., 이제, 사람, 만...     [걱정, 속상하, 의심]

[42694 rows x 2 columns]


### 어휘 표현 다양성 계산

In [45]:
def calculate_emotion_ratio(tokens):
    """
    감정 토큰의 고유 개수 / 전체 감정 토큰 수
    """
    unique = set(tokens)
    return len(unique) / len(tokens) if len(tokens) != 0 else 0

ratios = scripts_emotion.progress_apply(calculate_emotion_ratio)

# scripts 데이터프레임에 각종 정보 컬럼을 합쳐서 관리
scripts_text = scripts['text']
scripts = pd.DataFrame({
    'text': scripts_text,
    'tokens': scripts_token,
    'emotion_tokens': scripts_emotion,
    'emotion_ratios': ratios
})
print(scripts)

100%|██████████| 42694/42694 [00:00<00:00, 805583.81it/s]

                                                    text  \
0      지금까지 힘들게 일했는데 은퇴해서 돈이 없다고 하니 자식이 화를 내서 상처를 받았어...   
1      친구한테 은퇴할 거라고 얘기했더니 앞으로 뭘 먹고 살 거냐면서 비웃더라고. 기분이 ...   
2      친구한테 은퇴한다고 했더니 그게 말이나 되는 거냐며 날 한심한 사람 취급해서 서운했...   
3      그동안 열심히 달려와서 좀 쉬려고 하는데 은퇴한다고 하니 주변에서 다 말려서 기분이...   
4      많은 고민 후 은퇴를 결심했는데 주변에서 다들 섣부른 생각이라고 해서 마음이 안 좋...   
...                                                  ...   
42689  나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고....   
42690  몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나. 건강...   
42691  이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정...   
42692  몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가...   
42693  남편과 결혼한 지 사십 년이야. 이제 사람 만나는 것도 버겁고 알던 사람도 점점 사...   

                                                  tokens    emotion_tokens  \
0      [지금, 까지, 힘들, 게, 일하, 었, 는데, 은퇴, 하, 어서, 돈, 이, 없,...                []   
1      [친구, 한테, 은퇴, 하, ᆯ, 거, 이, 라고, 얘기, 하, 었, 더니, 앞, ...  [나쁘, 나쁘, 나쁘, 나쁘]   
2      [친구, 한테, 은퇴, 하, ᆫ다고, 하, 었, 더니, 그것, 이, 

# 척도 #3

### 감정 표현 밀도(강도) 계산

In [46]:
# 감정 토큰 수 / 전체 토큰 수로 정의
scripts['density'] = scripts['emotion_tokens'].apply(len) / scripts['tokens'].apply(len)

print(scripts)

                                                    text  \
0      지금까지 힘들게 일했는데 은퇴해서 돈이 없다고 하니 자식이 화를 내서 상처를 받았어...   
1      친구한테 은퇴할 거라고 얘기했더니 앞으로 뭘 먹고 살 거냐면서 비웃더라고. 기분이 ...   
2      친구한테 은퇴한다고 했더니 그게 말이나 되는 거냐며 날 한심한 사람 취급해서 서운했...   
3      그동안 열심히 달려와서 좀 쉬려고 하는데 은퇴한다고 하니 주변에서 다 말려서 기분이...   
4      많은 고민 후 은퇴를 결심했는데 주변에서 다들 섣부른 생각이라고 해서 마음이 안 좋...   
...                                                  ...   
42689  나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고....   
42690  몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나. 건강...   
42691  이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정...   
42692  몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가...   
42693  남편과 결혼한 지 사십 년이야. 이제 사람 만나는 것도 버겁고 알던 사람도 점점 사...   

                                                  tokens    emotion_tokens  \
0      [지금, 까지, 힘들, 게, 일하, 었, 는데, 은퇴, 하, 어서, 돈, 이, 없,...                []   
1      [친구, 한테, 은퇴, 하, ᆯ, 거, 이, 라고, 얘기, 하, 었, 더니, 앞, ...  [나쁘, 나쁘, 나쁘, 나쁘]   
2      [친구, 한테, 은퇴, 하, ᆫ다고, 하, 었, 더니, 그것, 이, 